In [1]:
import os

os.chdir('../')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data_train_path = "Dataset/data_train.csv"

URM_all_dataframe = pd.read_csv(data_train_path)
print(URM_all_dataframe.head(10))

   user_id  item_id  data
0        0        0   1.0
1        0        2   1.0
2        0      120   1.0
3        0      128   1.0
4        0      211   1.0
5        0      232   1.0
6        0      282   1.0
7        0      453   1.0
8        0      458   1.0
9        0      491   1.0


In [3]:
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout



URM_all = sps.coo_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))
URM_all = URM_all.tocsr()

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


In [4]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

slim_W = sps.load_npz("slim_best_W_TRAINVAL.npz")
easeR_W = sps.load_npz("easeR_best_W_TRAINVAL.npz")

alpha_slim_EASER = 0.622977719931239

similarity_SLIM_EASER = slim_W.multiply(alpha_slim_EASER) + easeR_W.multiply(1-alpha_slim_EASER)
del slim_W, easeR_W

import gc
gc.collect()

# RP3
rp3_recommender = RP3betaRecommender(URM_train_validation)
rp3_recommender.fit(topK=14, alpha=0.3683550822991944, beta=0.19877125816137325)

# Hybrid
final_recommender = ItemKNNCustomSimilarityRecommender(URM_train_validation)
alpha = 0.8407690672301905
new_similarity = similarity_SLIM_EASER.multiply(alpha) + rp3_recommender.W_sparse.multiply(1-alpha)
del similarity_SLIM_EASER
del rp3_recommender
import gc
gc.collect()
final_recommender.fit(new_similarity)


RP3betaRecommender: Similarity column 38121 (100.0%), 3370.42 column/sec. Elapsed time 11.31 sec


In [7]:
'''result,factors,alpha,regularization
0.04866350388073791,1903.0,10.365656056877015,0.43781149051372426
'''

from Recommenders.MatrixFactorization.ImplicitALSRecommender import ImplicitALSRecommender

als_recommender = ImplicitALSRecommender(URM_train_validation)
als_recommender.fit(factors=1903, regularization=0.43781149051372426, iterations=10, use_gpu=False, **{"alpha": 10.365656056877015})

  0%|          | 0/10 [00:00<?, ?it/s]

In [14]:
from Recommenders.Hybrid.GeneralizedLinearCoupleHybridRecommender import GeneralizedLinearCoupleHybridRecommender
import optuna

recommenders = [final_recommender, als_recommender]
def objective_function(trial):
    recommender = GeneralizedLinearCoupleHybridRecommender(URM_train_validation, recommenders)
    alpha = trial.suggest_uniform('alpha', 0, 1)
    recommender.fit(alpha)
    result_dict, _ = evaluator_test.evaluateRecommender(recommender)
    MAP = result_dict.loc[10]["MAP"]
    return MAP

class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/Hybrid/hybrid_best_IALS_LINEAR.csv", index = False)

optuna_study = optuna.create_study(study_name="HYBRID_IALS_INCLUDED", direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 100,
                      )

[I 2024-12-30 01:19:46,526] A new study created in memory with name: HYBRID_IALS_INCLUDED
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_98650/1578609616.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0, 1)


EvaluatorHoldout: Processed 35595 (100.0%) in 4.91 min. Users per second: 121


[I 2024-12-30 01:24:41,054] Trial 0 finished with value: 0.050479412348352994 and parameters: {'alpha': 0.097905282755954}. Best is trial 0 with value: 0.050479412348352994.
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_98650/1578609616.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0, 1)


EvaluatorHoldout: Processed 35595 (100.0%) in 4.40 min. Users per second: 135


[I 2024-12-30 01:29:05,538] Trial 1 finished with value: 0.05936781628416496 and parameters: {'alpha': 0.5912161468947038}. Best is trial 1 with value: 0.05936781628416496.
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_98650/1578609616.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0, 1)


...

[I 2024-12-30 08:03:46,503] Trial 97 finished with value: 0.061384563330410584 and parameters: {'alpha': 0.9357885551998323}. Best is trial 96 with value: 0.061388178739209306.
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_98650/1578609616.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0, 1)


EvaluatorHoldout: Processed 35595 (100.0%) in 4.04 min. Users per second: 147


[I 2024-12-30 08:07:49,245] Trial 98 finished with value: 0.06136303810281713 and parameters: {'alpha': 0.9329867348614059}. Best is trial 96 with value: 0.061388178739209306.
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_98650/1578609616.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0, 1)


EvaluatorHoldout: Processed 35595 (100.0%) in 3.94 min. Users per second: 151


[I 2024-12-30 08:11:45,529] Trial 99 finished with value: 0.06128886034911959 and parameters: {'alpha': 0.9765662144824331}. Best is trial 96 with value: 0.061388178739209306.


In [15]:
best_index = save_results.results_df["result"].idxmax()
best_hyperparams = save_results.results_df.loc[best_index].to_dict()
print(best_hyperparams)

{'result': 0.061388178739209306, 'alpha': 0.9363330179334278}


# Preparing submission

In [17]:
alpha = best_hyperparams["alpha"]

gc.collect()
del new_similarity, final_recommender, als_recommender, recommenders

In [4]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender
from Recommenders.MatrixFactorization.ImplicitALSRecommender import ImplicitALSRecommender
from Recommenders.Hybrid.GeneralizedLinearCoupleHybridRecommender import GeneralizedLinearCoupleHybridRecommender


slim_W = sps.load_npz("slim_best_W.npz")
easeR_W = sps.load_npz("easeR_best_W.npz")
alpha_slim_EASER = 0.622977719931239

similarity_SLIM_EASER = slim_W.multiply(alpha_slim_EASER) + easeR_W.multiply(1-alpha_slim_EASER)
del slim_W, easeR_W
# RP3
rp3_recommender = RP3betaRecommender(URM_all)
rp3_recommender.fit(topK=14, alpha=0.3683550822991944, beta=0.19877125816137325)

# Hybrid
final_recommender = ItemKNNCustomSimilarityRecommender(URM_all)
alpha = 0.8407690672301905
new_similarity = similarity_SLIM_EASER.multiply(alpha) + rp3_recommender.W_sparse.multiply(1-alpha)
del similarity_SLIM_EASER, rp3_recommender
final_recommender.fit(new_similarity)

RP3betaRecommender: Similarity column 38121 (100.0%), 2480.80 column/sec. Elapsed time 15.36 sec


In [5]:
als_recommender = ImplicitALSRecommender(URM_all)
als_recommender.fit(factors=1903, regularization=0.43781149051372426, iterations=10, use_gpu=False, **{"alpha": 10.365656056877015})

/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/10 [00:00<?, ?it/s]

In [6]:
alpha = 0.9363330179334278

recommenders = [final_recommender, als_recommender]

linear_comb_rec = GeneralizedLinearCoupleHybridRecommender(URM_all, recommenders)
linear_comb_rec.fit(alpha)

In [8]:
target_user = pd.read_csv("Dataset/data_target_users_test.csv")
print(target_user.head(10))

user_id = target_user['user_id']
recommendations = []
for user in user_id:
    recommendations.append(linear_comb_rec.recommend(user, cutoff=10))

# Saving in a submission file for the competition
submission = pd.DataFrame(columns=['user_id', 'item_list'])
submission['user_id'] = user_id
# Convert list of recommendations into a space-separated string
submission['item_list'] = [' '.join(map(str, recs)) for recs in recommendations]

# Save the submission file
submission.to_csv(f"submissions/hybrid_best_IALS_alpha:{alpha}.csv", index=False)

   user_id
0        0
1        1
2        2
3        3
4        4
5        5
6        6
7        7
8        8
9        9
